In [18]:
import os
import json
import pandas as pd
from pathlib import Path

# Define the path to the runs folder
runs_folder = Path('runs')

# Initialize an empty list to store the data
data = []

# Iterate through each file in the runs folder
for file_name in os.listdir(runs_folder):
    if file_name.endswith('.json'):
        file_path = runs_folder / file_name
        with open(file_path, 'r') as file:
            content = json.load(file)
            # Extract dataset and seed from the filename
            parts = file_name.split('_')
            dataset = parts[3].lower()
            seed = parts[-1].split('.')[0]
            mcc = content.get('eval_matthews_correlation', 0)
            data.append({'Dataset': dataset, 'seed': seed, 'MCC': mcc})

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Convert seed to integer for proper sorting
df['seed'] = df['seed'].astype(int)

# Sort the DataFrame by Dataset and then by seed
df = df.sort_values(by=['Dataset', 'seed'])

# Compute the average MCC for each dataset
average_mcc = df.groupby('Dataset')['MCC'].mean().reset_index()
average_mcc.rename(columns={'MCC': 'Average MCC'}, inplace=True)

# Convert MCC to percentage by multiplying by 100, rounding to 2 decimal places, and adding percentage sign
df['MCC'] = df['MCC'].apply(lambda x: f"{(x * 100):.2f}%")
average_mcc['Average MCC'] = average_mcc['Average MCC'].apply(lambda x: f"{(x * 100):.2f}%")


## Sorted Runs Table

In [16]:
print(df.to_markdown(index=False))

| Dataset   |   seed | MCC    |
|:----------|-------:|:-------|
| all       |     15 | 91.64% |
| all       |     44 | 90.95% |
| all       |     48 | 91.05% |
| all       |     65 | 89.07% |
| all       |    172 | 90.54% |
| notata    |     31 | 93.03% |
| notata    |     42 | 93.13% |
| notata    |     77 | 92.24% |
| tata      |     11 | 63.76% |
| tata      |     53 | 61.89% |
| tata      |     80 | 60.48% |


## Average MCC per Dataset

In [17]:
print(average_mcc.to_markdown(index=False))

| Dataset   | Average MCC   |
|:----------|:--------------|
| all       | 90.65%        |
| notata    | 92.80%        |
| tata      | 62.05%        |


In [19]:
expected_values = {
    "all": 0.9048,
    "notata": 0.9305,
    "tata": 0.6156
}

# Convert the percentage strings back to float values for calculations
average_mcc_values = average_mcc.copy()
average_mcc_values['Average MCC'] = average_mcc_values['Average MCC'].str.rstrip('%').astype(float) / 100

# Create a comparison dataframe
comparison = pd.DataFrame({
    'Dataset': list(expected_values.keys()),
    'Our MCC': [average_mcc_values[average_mcc_values['Dataset'] == dataset]['Average MCC'].values[0] for dataset in expected_values.keys()],
    'Expected MCC': list(expected_values.values())
})

# Calculate absolute and relative errors
comparison['Absolute Error'] = abs(comparison['Our MCC'] - comparison['Expected MCC'])
comparison['Relative Error (%)'] = comparison['Absolute Error'] / comparison['Expected MCC'] * 100

# Format values for display
comparison['Our MCC'] = comparison['Our MCC'].apply(lambda x: f"{(x * 100):.2f}%")
comparison['Expected MCC'] = comparison['Expected MCC'].apply(lambda x: f"{(x * 100):.2f}%")
comparison['Absolute Error'] = comparison['Absolute Error'].apply(lambda x: f"{(x * 100):.2f}%")
comparison['Relative Error (%)'] = comparison['Relative Error (%)'].apply(lambda x: f"{x:.2f}%")

# Display the comparison table using to_markdown
print(comparison.to_markdown(index=False))

| Dataset   | Our MCC   | Expected MCC   | Absolute Error   | Relative Error (%)   |
|:----------|:----------|:---------------|:-----------------|:---------------------|
| all       | 90.65%    | 90.48%         | 0.17%            | 0.19%                |
| notata    | 92.80%    | 93.05%         | 0.25%            | 0.27%                |
| tata      | 62.05%    | 61.56%         | 0.49%            | 0.80%                |
